In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#install CATBOOST
pip install catboost

     |████████████████████████████████| 62.9MB 37kB/s 


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as seabornInstance 
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score

# dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
# dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))
# temp=pd.concat([dataset,dataWithoutLabels], ignore_index=True)
# print(dataset.tail)
# print(temp.columns.tolist())
#temp['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
#print(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].unique())

In [0]:
# col='Yearly Income in addition to Salary (e.g. Rental Income)'
# temp[col]=temp[col].map(lambda x: x.rstrip(' EUR'))
# print(temp[col].unique())
# a=temp[col].value_counts(dropna=False)
# print(a)
# X_train,X_test = temp.iloc[:1048574],temp.iloc[1048575:]
# print(X_train.head)
# rename_cols = {"Income in EUR":'Income'}
# train = train.rename(columns=rename_cols)
# for col in fill_col_dict.keys():
    # data[col] = data[col].fillna(fill_col_dict[col])
#X_train,X_test = data[features_col].iloc[:1048574],data[features_col].iloc[1048575:]

In [0]:

# to add noise on the data if dataframe has any NAN values
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


#Function to apply Target encoding on categorial data it takes train and test column along with target column to give logical mean value to strings.

def target_encode(trn_series, tst_series,target):
    min_samples_leaf=1
    smoothing=1
    noise_level=0
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # Apply averages to train and test series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)


In [0]:

def prepareData(dataset,dataWithoutLabels):
  #training data+test data
  
  dataset['Gender']=dataset['Gender'].replace(['unknown','0',np.nan],'UNKOWN')  
  dataset['Gender']=dataset['Gender'].replace(['f'],'female')  
  dataset['Housing Situation']=dataset['Housing Situation'].replace(['nA',0,'0'],'UNKOWN')
  dataset['Work Experience in Current Job [years]']=pd.to_numeric(dataset['Work Experience in Current Job [years]'].replace(['#NUM!'],0))
  dataset['Satisfation with employer']=dataset['Satisfation with employer'].replace([np.nan],'Somewhat Happy')
  dataset['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataset['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataset['Hair Color']=dataset['Hair Color'].replace(['unknown','0',np.nan],'UNKOWN')
  #print('fillna mean')
  #dataset=dataset.fillna(dataset.mean())
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['unknown','0',np.nan],'UNKOWN')  
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['f'],'female')  
  dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['nA',0,'0'],'UNKOWN')
  dataWithoutLabels['Work Experience in Current Job [years]']=pd.to_numeric(dataWithoutLabels['Work Experience in Current Job [years]'].replace(['#NUM!'],0))
  dataWithoutLabels['Satisfation with employer']=dataWithoutLabels['Satisfation with employer'].replace([np.nan],'Somewhat Happy')
  dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataWithoutLabels['Hair Color']=dataWithoutLabels['Hair Color'].replace(['unknown','0',np.nan],'UNKOWN')
  dataWithoutLabels=dataWithoutLabels.fillna(method='ffill')
  dataset=dataset.fillna(method='ffill')
    #encoding of categorical data
  dataset['Housing Situation'],dataWithoutLabels['Housing Situation']=target_encode(dataset['Housing Situation'], dataWithoutLabels['Housing Situation'],dataset['Total Yearly Income [EUR]'])
  dataset['Satisfation with employer'],dataWithoutLabels['Satisfation with employer']=target_encode(dataset['Satisfation with employer'], dataWithoutLabels['Satisfation with employer'],dataset['Total Yearly Income [EUR]'])
  dataset['Gender'],dataWithoutLabels['Gender']=target_encode(dataset['Gender'], dataWithoutLabels['Gender'],dataset['Total Yearly Income [EUR]'])
  dataset['Country'],dataWithoutLabels['Country']=target_encode(dataset['Country'], dataWithoutLabels['Country'],dataset['Total Yearly Income [EUR]'])
  dataset['Profession'],dataWithoutLabels['Profession']=target_encode(dataset['Profession'], dataWithoutLabels['Profession'],dataset['Total Yearly Income [EUR]'])
  dataset['University Degree'],dataWithoutLabels['University Degree']=target_encode(dataset['University Degree'], dataWithoutLabels['University Degree'],dataset['Total Yearly Income [EUR]'])
  dataset['Hair Color'],dataWithoutLabels['Hair Color']=target_encode(dataset['Hair Color'], dataWithoutLabels['Hair Color'],dataset['Total Yearly Income [EUR]'])
  return dataset,dataWithoutLabels


In [0]:

  # plot the Values on a graph
def plotActualVsPredictedIncome(y_test, y_pred):
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    df1 = df.head(25)
    df1.plot(kind='bar',figsize=(10,8))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()    

def trainModel(X_train, y_train, X_test, y_test):
   
    
    #Implementing linear regression (error rate 110000+)
    # trainingRegressor = LinearRegression()  
    # trainingRegressor.fit(X_train, y_train)
    # y_pred = trainingRegressor.predict(X_test)
    
    
    #Implementing RandomForest Regressor (error rate 80000+)
    trainingRegressor = RandomForestRegressor(n_estimators=100)
    trainingRegressor.fit(X_train,y_train)
    y_pred = trainingRegressor.predict(X_test)
    
    
    
    #Implementing caatboost (error rate 60000+)
    #learning rate to get an optimal solution 
    
    # trainingRegressor = CatBoostRegressor(iterations=7000,learning_rate=0.02)
    # trainingRegressor.fit(X_train, y_train,use_best_model=True,verbose=True)
    # y_pred=trainingRegressor.predict(X_test)
    
    
    #evaluating the error
    df = pd.DataFrame({'Test': y_test, 'Prediction': y_pred})
    print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
    
    #ploting the values on the graph
    #plotActualVsPredictedIncome(y_test, y_pred)

    return trainingRegressor
 

def runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels):
    print(featuresToConsider)
    print('dataset',dataWithoutLabels[featuresToConsider].dtypes)
    #get the values for X_test    
    X = dataWithoutLabels[featuresToConsider].values
    print('head',X)
    #prediciting the values on the real test data
    predictedData = regressor.predict(X)
    df = pd.DataFrame({'Predicted': predictedData})

    #write the data to csv
    fileName = '/content/drive/My Drive/Colab Notebooks/ML/RF_Sumission_file.csv'
    df.to_csv(fileName, sep=',',index=False)
    
    

    

#Reading data from CSV
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))
#combined_data=pd.concat([dataset,dataWithoutLabels], ignore_index=True)

#preprocessing data on both the datasets



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:

dataset,dataWithoutLabels = prepareData(dataset,dataWithoutLabels)


In [0]:
featuresToConsider = ['Year of Record', 'Housing Situation', 'Crime Level in the City of Employement', 'Work Experience in Current Job [years]', 'Satisfation with employer', 'Gender', 'Age', 'Country', 'Size of City', 'Profession', 'University Degree', 'Wears Glasses', 'Hair Color', 'Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)']

#getting thte target data
y = dataset['Total Yearly Income [EUR]'].values
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

#Normalization
#dataset[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataset[featuresToConsider]), columns=featuresToConsider)
#dataWithoutLabels[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataWithoutLabels[featuresToConsider]), columns=featuresToConsider)
# print(dataset[featuresToConsider].dtypes)
# print(dataset[featuresToConsider].dtypes)
# print(dataset['Year of Record'].unique())

In [0]:

#Features Dataset
X = dataset[featuresToConsider].values

#divide the training data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

#get the trained model
regressor = trainModel(X_train, y_train, X_test, y_test)
# print(dataWithoutLabels.head)
# print(dataWithoutLabels.isnull().sum())
#run on the real test data using the. trained model
runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels)

Mean squared error: 99.81
['Year of Record', 'Housing Situation', 'Crime Level in the City of Employement', 'Work Experience in Current Job [years]', 'Satisfation with employer', 'Gender', 'Age', 'Country', 'Size of City', 'Profession', 'University Degree', 'Wears Glasses', 'Hair Color', 'Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)']
dataset Year of Record                                              float64
Housing Situation                                           float64
Crime Level in the City of Employement                        int64
Work Experience in Current Job [years]                      float64
Satisfation with employer                                   float64
Gender                                                      float64
Age                                                           int64
Country                                                     float64
Size of City                                                  int64
Profession 